In [2]:
import pandas as pd

Membuat EDA untuk Dataset nya terlebih dahulu

In [3]:
musik = pd.read_csv('D:/PENS-EEPIS/SDT A Semester 2 2025/Pemrosesan Data/CBF/Cleaning & Merge Data/top_hits.csv')
musik

,song,artis,album,viewed,tahun_rilis,genre
0,Semua Aku Dirayakan,Nadin Amizah,Semua Aku Dirayakan,190277503,2023,indie
1,Hujan Mata Pisau,Fstvlst,Hits Kitsch,3523565,2014,indie
2,"the more i try to trace you forthwith, the les...",eleventwelfth,SIMILAR,4173966,2023,indie
3,Rumah Ke Rumah,Hindia,Menari Dengan Bayangan,469951327,2019,indie
4,Asmalibrasi,Soegi Bornean,Asmalibrasi,229593278,2019,indie
...,...,...,...,...,...,...
156,Trully Kawan,Steven & Coconuttreez,Good Atmosphere,5546961,2008,reggae
157,Selamat Jalan Kawan,Steven & Coconuttreez,Good Atmosphere,2505444,2008,reggae
158,Hingga Ku Jenuh,Steven & Coconuttreez,Good Atmosphere,4734568,2008,reggae
159,PHP,Souljah,This Is Souljah,4602730,2014,reggae


In [4]:
musik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   song         161 non-null    object
 1   artis        161 non-null    object
 2   album        161 non-null    object
 3   viewed       161 non-null    int64 
 4   tahun_rilis  161 non-null    int64 
 5   genre        161 non-null    object
dtypes: int64(2), object(4)
memory usage: 7.7+ KB


# 1. LOAD DATA

In [5]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Buat DataFrame
df = pd.read_csv("D:/PENS-EEPIS/SDT A Semester 2 2025/Pemrosesan Data/CBF/Cleaning & Merge Data/top_hits.csv")
print(f"Data loaded: {len(df)} songs")
print("\nDataset Overview:")
print(df.head())
print(f"\nDataset Shape: {df.shape}")


Data loaded: 161 songs

Dataset Overview:
                                                song          artis  \
0                                Semua Aku Dirayakan   Nadin Amizah   
1                                   Hujan Mata Pisau        Fstvlst   
2  the more i try to trace you forthwith, the les...  eleventwelfth   
3                                     Rumah Ke Rumah         Hindia   
4                                        Asmalibrasi  Soegi Bornean   

                    album     viewed  tahun_rilis  genre  
0     Semua Aku Dirayakan  190277503         2023  indie  
1             Hits Kitsch    3523565         2014  indie  
2                 SIMILAR    4173966         2023  indie  
3  Menari Dengan Bayangan  469951327         2019  indie  
4             Asmalibrasi  229593278         2019  indie  

Dataset Shape: (161, 6)


# 2. EXPLORASI DATA SEDERHANA

In [6]:
print("\nQuick Data Exploration:")
print(f"Genres: {df['genre'].unique()}")
print(f"Year range: {df['tahun_rilis'].min()} - {df['tahun_rilis'].max()}")
print(f"Total artists: {df['artis'].nunique()}")


Quick Data Exploration:
Genres: ['indie' 'pop' 'rock' 'reggae']
Year range: 1993 - 2025
Total artists: 71


# 3. PREPROCESSING DATA


In [7]:
print("\nPreprocessing data...")

# Gabungkan fitur untuk similarity (genre + artis)
df['combined_features'] = df['genre'] + ' ' + df['artis'].str.lower()

# Normalisasi fitur numerik sederhana
scaler = StandardScaler()
df['tahun_norm'] = scaler.fit_transform(df[['tahun_rilis']])
df['viewed_norm'] = scaler.fit_transform(df[['viewed']])


Preprocessing data...


# 4. BUAT TF-IDF MATRIX


In [8]:
stopwords_indonesia = [
    'yang', 'dan', 'di', 'ke', 'dari', 'adalah', 'untuk', 'dengan',
    'pada', 'ini', 'itu', 'atau', 'karena', 'juga', 'oleh', 'sebagai',
    'akan', 'tidak', 'sudah', 'masih', 'bagi', 'dalam', 'terhadap', 'lebih', 'agar'
    # Tambahkan sendiri sesuai kebutuhan
]

# Vectorizer untuk combined features
tfidf = TfidfVectorizer(stop_words=stopwords_indonesia)
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

# 5. HITUNG COSINE SIMILARITY


In [9]:
# Hitung cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix)

print(f"Similarity matrix: {cosine_sim.shape}")

Similarity matrix: (161, 161)


# 6. FUNGSI REKOMENDASI SEDERHANA


In [10]:
def recommend_songs(song_title, num_rec=5):
    """
    Fungsi untuk mendapatkan rekomendasi lagu berdasarkan judul
    """
    # Cari index lagu
    try:
        idx = df[df['song'].str.lower() == song_title.lower()].index[0]
        print(f"Found song: {df.iloc[idx]['song']} by {df.iloc[idx]['artis']}")
    except IndexError:
        print(f"Song '{song_title}' not found!")
        print("Available songs:")
        for i, song in enumerate(df['song'][:10]):  # Menampilkan 10 lagu pertama sebagai contoh
            print(f"  - {song}")
        return None
    
    # Ambil similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort berdasarkan similarity (tinggi ke rendah)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Ambil top N (skip index 0 = lagu itu sendiri)
    top_songs = sim_scores[1:num_rec+1]
    
    # Buat list rekomendasi
    recommendations = []
    for i, score in top_songs:
        rec = {
            'song': df.iloc[i]['song'],
            'artis': df.iloc[i]['artis'],
            'album': df.iloc[i]['album'],
            'genre': df.iloc[i]['genre'],
            'tahun': df.iloc[i]['tahun_rilis'],
            'views': df.iloc[i]['viewed'],
            'similarity': score
        }
        recommendations.append(rec)
    
    return recommendations

# 7. TAMPILKAN REKOMENDASI


In [11]:
def show_recommendations(song_title, num_rec=5):
    """
    Tampilkan rekomendasi dengan format rapi
    """
    print(f"\nRECOMMENDATIONS FOR: '{song_title}'")
    
    recs = recommend_songs(song_title, num_rec)
    
    if recs:
        for i, rec in enumerate(recs, 1):
            print(f"\n{i}. 🎵 {rec['song']}")
            print(f"   Artist: {rec['artis']}")
            print(f"   Album: {rec['album']}")
            print(f"   Genre: {rec['genre']}")
            print(f"   Year: {rec['tahun']}")
            print(f"   Views: {rec['views']:,}")
            print(f"   Similarity: {rec['similarity']:.3f}")

    else:
        print("No recommendations generated.")

# 8. TESTING SISTEM

In [12]:
# Test 1: Indie song
print("\n[TEST 1] Indie Song Recommendation:")
show_recommendations('Semua Aku Dirayakan', 10)

# Test 2: Reggae song  
print("\n[TEST 2] Pop Song Recommendation:")
show_recommendations('Rumah Ke Rumah', 5)


[TEST 1] Indie Song Recommendation:

RECOMMENDATIONS FOR: 'Semua Aku Dirayakan'
Found song: Semua Aku Dirayakan by Nadin Amizah

1. 🎵 Bertaut
   Artist: Nadin Amizah
   Album: Selamat Ulang Tahun
   Genre: indie
   Year: 2020
   Views: 503,334,418
   Similarity: 1.000

2. 🎵 Taruh
   Artist: Nadin Amizah
   Album: Selamat Ulang Tahun
   Genre: indie
   Year: 2020
   Views: 114,823,029
   Similarity: 1.000

3. 🎵 Rayuan Perempuan Gila
   Artist: Nadin Amizah
   Album: Rayuan Perempuan Gila
   Genre: indie
   Year: 2023
   Views: 228,372,657
   Similarity: 1.000

4. 🎵 Rumah Ke Rumah
   Artist: Hindia
   Album: Menari Dengan Bayangan
   Genre: indie
   Year: 2019
   Views: 469,951,327
   Similarity: 0.190

5. 🎵 Kita Ke Sana
   Artist: Hindia
   Album: Lagipula Hidup Akan Berakhir
   Genre: indie
   Year: 2023
   Views: 90,998,925
   Similarity: 0.190

6. 🎵 Cincin
   Artist: Hindia
   Album: Lagipula Hidup Akan Berakhir
   Genre: indie
   Year: 2023
   Views: 227,594,270
   Similarity: 0.19

# 9. SIMPLE EVALUATION


In [13]:
def evaluate_genre_similarity():
    """
    Evaluasi sederhana: cek apakah rekomendasi punya genre yang sama
    """
    print("\n📈 SIMPLE EVALUATION")
    print("=" * 30)
    
    test_songs = ['Semua Aku Dirayakan', 'Truly Kawan']
    
    for song_title in test_songs:
        try:
            # Get original genre
            original_idx = df[df['song'].str.lower() == song_title.lower()].index[0]
            original_genre = df.iloc[original_idx]['genre']
            
            # Get recommendations
            recs = recommend_songs(song_title, 3)
            
            if recs:
                same_genre = sum(1 for rec in recs if rec['genre'] == original_genre)
                print(f"🎵 {song_title} ({original_genre}):")
                print(f"   Same genre recommendations: {same_genre}/3")
                print(f"   Success rate: {same_genre/3*100:.1f}%")
        except:
            continue



In [14]:
# Run evaluation
evaluate_genre_similarity()

print("\n✅ RECOMMENDATION SYSTEM COMPLETED!")


📈 SIMPLE EVALUATION
Found song: Semua Aku Dirayakan by Nadin Amizah
🎵 Semua Aku Dirayakan (indie):
   Same genre recommendations: 3/3
   Success rate: 100.0%

✅ RECOMMENDATION SYSTEM COMPLETED!
